## Simple CNN for Edgelovers


In this notebook you train a very simple CNN with only 1 kernel to discriminate images containing vertical (y=0) from those containing horizontal stripes (y=1). You can use keras for the solution.

![](ch02_2.20_kernel.png)

### a)  Generation of the data
Write a function which creates an artficially dataset of greyscale images (50x50 pixel) with 10 vertical or horizontal bars (10 pixel long). Use this function to create a training and validation dataset of 1000 examples each.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import layers, models
from random import random


# Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = False
plt.gray()

In [ ]:
def generate_image():
    
    image = np.random.rand(50, 50)
    vertical = random() > 0.5
    index = np.array(np.arange(0, 50))
    np.random.shuffle(index)
    
    # place either 10 horizontal or vertical bars at random positions in the image
    if vertical:
        for i in range(0, 10):
            start = np.random.randint(image.shape[1] - 9)
            image[start:start + 10, index[i]] = 0
        return image, 0
    else:
        for i in range(0, 10):
            start = np.random.randint(image.shape[1] - 9)
            image[index[i], start: start + 10] = 0
        return image, 1

In [ ]:
def generate_dataset(n):
    
    x_arr = []
    y_arr = []
    
    for i in range(0, n):
        x, y = generate_image()
        x_arr.append(x)
        y_arr.append(y)
    
    return np.array(x_arr), np.array(y_arr)

In [ ]:
# generate training set
train_x, train_y = generate_dataset(1000)

# generate validation set
test_x, test_y = generate_dataset(1000)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_x[i])
    plt.xlabel(train_y[i])
plt.show()

### b) Build the simplest possible CNN
Make a CNN with one convolution (5x5, haveing two outputs (one for horizontal, one for vertical). Train the CNN on the data from a). You should have no more than 30 trainable parameters in the network. 
* Plot the learning curves: (epochs vs training loss and validation loss) and epochs vs accuracy.You should get an accuracy of approximatly 1.

Hint: Use the max-pooling operation in a clever way.

In [ ]:
model = models.Sequential([
    layers.Conv2D(1, (5, 5), activation='relu', input_shape=(50, 50, 1)),
    layers.MaxPool2D((46, 46)),
    layers.Flatten(),
    layers.Dense(2, activation='softmax')
])
model.summary()

In [ ]:
train_y = keras.utils.to_categorical(train_y, num_classes=2)
test_y = keras.utils.to_categorical(test_y)

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.03),
            loss=keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20, 
                    validation_data=(test_x, test_y))

In [ ]:
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
test_loss, test_acc = model.evaluate(test_x, test_y)
print(test_acc)

### c) Visualize the learned kernel
Visualize the learned kernel, you might want to use `model.get_weights()`. Does the learned kernel makes sense?

In [ ]:
kernel = model.get_weights()[0]

In [ ]:
#plot learned kernel
plt.imshow(np.array(np.reshape(kernel, (5, 5))))